In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


In [ ]:
def build_descriptive_text_vocab(input_text):
  vocab = set()
  vocab.add("<UNK>")
  for text in input_text:
    for word in text.split():
      word = remove_punctuation(word)
      if word:
        vocab.add(word)
  return {token: i for i, token in enumerate(vocab)}

def vectorize_descriptive_text(input_text, vocab):
  vectorized_text = np.zeros(len(vocab))
  for word in input_text.split():
    word = remove_punctuation(word)
    if word in vocab:
      vectorized_text[vocab[word]] += 1
    else:
      vectorized_text[vocab["<UNK>"]] += 1
  return vectorized_text

def remove_punctuation(word):
    punctuation = set([".", "(", ")", ",", ";", "?", "!", '"', ":", "'"])
    while word and word[0] in punctuation:
        word = word[1:]
    while word and word[-1] in punctuation:
        word = word[:-1]
    return word.lower()




class CustomCSVDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (str): Path to the CSV file containing the data.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)  # Load the CSV file
        self.transform = transform

        # Assuming the last column is the label
        self.features = self.data.iloc[:, :-1].values  # All columns except the last one (features)
        self.labels = self.data.iloc[:, -1].values    # The last column (label)

        # Standardize features if necessary
        self.scaler = StandardScaler()
        self.features = self.scaler.fit_transform(self.features)  # Normalize features

    def __len__(self):
        """Returns the total number of samples."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to fetch.
        Returns:
            tuple: (features, label) where features are the input tensor and label is the output tensor.
        """
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        if self.transform:
            features = self.transform(features)

        return features, label
